In [1]:
from main import process_burndown_rs, generate_df_pool_process

In [2]:
n_jobs = 10
no_of_issues = 1_000_000
resolution_type = "hours"
resolution_val = 1
max_workers = 22

In [3]:
low1 = 5
high1 = 3
w1 =1
d1 =8
h1 =8
m1 =3
s1 =1

In [4]:
df_pool = generate_df_pool_process(n_jobs, no_of_issues, max_workers, low1, high1, w1, d1, h1, m1, s1)

Pool generation 0 finished
Pool generation 5 finished


In [5]:
import plotly.express as px

In [6]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd

In [7]:
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_burndown_rs, resolution_type, resolution_val, df, thread) for thread, df in enumerate(df_pool)]
    res_list = []
    dates_list = []
    hits_list = []
    for future in as_completed(futures):
        res, dates, hits = future.result()
        res_list += [res for _ in hits]
        dates_list += list(dates)
        hits_list += list(hits)
        print(f"Thread number {res} as completed its task")
    final_results = {
        "n_job": res_list,
        "date": dates_list,
        "hit": hits_list,
    }
df_res = pd.DataFrame(final_results)

Thread number 9 as completed its task
Thread number 2 as completed its task
Thread number 3 as completed its task
Thread number 6 as completed its task
Thread number 8 as completed its task
Thread number 1 as completed its task
Thread number 5 as completed its task
Thread number 0 as completed its task
Thread number 7 as completed its task
Thread number 4 as completed its task


In [8]:
df_res

,n_job,date,hit
0,9,2025-06-15T19:20:10.013130+00:00,0
1,9,2025-06-15T19:20:10.013130+00:00,405
2,9,2025-06-15T20:20:10.013130+00:00,405
3,9,2025-06-15T20:20:10.013130+00:00,808
4,9,2025-06-15T21:20:10.013130+00:00,808
...,...,...,...
38195,4,2025-09-03T07:20:09.722177+00:00,1
38196,4,2025-09-03T08:20:09.722177+00:00,1
38197,4,2025-09-03T08:20:09.722177+00:00,1
38198,4,2025-09-03T09:20:09.722177+00:00,1


In [9]:
df_res_1 = df_res.loc[df_res.n_job == 1]
df_res_1

,n_job,date,hit
19096,1,2025-06-15T19:20:10.567746+00:00,0
19097,1,2025-06-15T19:20:10.567746+00:00,416
19098,1,2025-06-15T20:20:10.567746+00:00,416
19099,1,2025-06-15T20:20:10.567746+00:00,818
19100,1,2025-06-15T21:20:10.567746+00:00,818
...,...,...,...
22909,1,2025-09-03T05:20:10.567746+00:00,3
22910,1,2025-09-03T06:20:10.567746+00:00,3
22911,1,2025-09-03T06:20:10.567746+00:00,1
22912,1,2025-09-03T07:20:10.567746+00:00,1


In [10]:
fig = px.line(df_res_1, x="date", y="hit", title="Hit Counts Over Time")
fig.update_layout(width=1500, height=500)
fig.show()